## Тестирование датасета MOT20_ext

In [11]:
from src.data import MOT20ExtDataset
import numpy as np
import functools

In [15]:
dataset = MOT20ExtDataset('data/MOT20_ext/train/MOT20-01/')

### Полное тестирование 

In [17]:
len(dataset)

2301

### Фильтрация

In [4]:
df = dataset.ground_truth
df

,frame,id,bb_left,bb_top,bb_width,bb_height,is_consider,class,visibility
391,54,2,455,808,113,271,1,1,1.0
392,55,2,457,808,113,272,1,1,1.0
393,56,2,459,806,113,274,1,1,1.0
394,57,2,461,804,113,276,1,1,1.0
395,58,2,463,802,114,278,1,1,1.0
...,...,...,...,...,...,...,...,...,...
18529,425,61,969,517,61,139,1,1,1.0
18530,426,61,970,518,61,139,1,1,1.0
18531,427,61,972,519,61,139,1,1,1.0
18532,428,61,973,520,61,139,1,1,1.0


In [5]:
len(dataset)
len(dataset)
len(dataset)

2301

In [6]:
df = df[df['is_consider'] == 1]
df

,frame,id,bb_left,bb_top,bb_width,bb_height,is_consider,class,visibility
391,54,2,455,808,113,271,1,1,1.0
392,55,2,457,808,113,272,1,1,1.0
393,56,2,459,806,113,274,1,1,1.0
394,57,2,461,804,113,276,1,1,1.0
395,58,2,463,802,114,278,1,1,1.0
...,...,...,...,...,...,...,...,...,...
18529,425,61,969,517,61,139,1,1,1.0
18530,426,61,970,518,61,139,1,1,1.0
18531,427,61,972,519,61,139,1,1,1.0
18532,428,61,973,520,61,139,1,1,1.0


In [7]:
test_visibility = 0.95

In [8]:
df = df[df['visibility'] >= test_visibility]
df

,frame,id,bb_left,bb_top,bb_width,bb_height,is_consider,class,visibility
391,54,2,455,808,113,271,1,1,1.0
392,55,2,457,808,113,272,1,1,1.0
393,56,2,459,806,113,274,1,1,1.0
394,57,2,461,804,113,276,1,1,1.0
395,58,2,463,802,114,278,1,1,1.0
...,...,...,...,...,...,...,...,...,...
18529,425,61,969,517,61,139,1,1,1.0
18530,426,61,970,518,61,139,1,1,1.0
18531,427,61,972,519,61,139,1,1,1.0
18532,428,61,973,520,61,139,1,1,1.0


### Расчет длины

In [9]:
test_len = 1
type(test_len) == int

True

In [10]:
objects = df[df['id'] == 4]
print(objects['frame'].values)

[21 22 23 24 25 26]


In [11]:
def aggregate(x, y):
    if (not type(x) == list):
        if (y - x == 1):
            return [[x, y]]
        else:
            return [[x], [y]]
    else:
        last = x[-1][-1]
        if (y - last > 1):
            x.append([y])
        else:
            x[-1].append(y)
    return x

In [12]:
def split_to_continuous_segments(array):
    """Возвращает список непрерывных отрезков чисел"""
    if (len(array) == 0):
        return [[]]
    elif (len(array) == 1):
        return [array]
    else:
        return functools.reduce(aggregate, sorted(array))

In [13]:
split_to_continuous_segments([2, 23, 4, 6, 5, 1])

[[1, 2], [4, 5, 6], [23]]

In [14]:
def get_possible_tuples_count_segment(distance: int, segment: list[int]) -> int:
    return len(segment) - distance - 1

In [15]:
def get_neighbours_tuples_count(distance: int, segments: list[list[int]]):
    sum = 0
    prev = None
    for s in segments:
        if (prev == None):
            prev = s
            continue
        if (s[0] - prev[-1] - 1 == distance):
            sum += 1
        prev = s

    return sum

In [16]:
def get_possible_tuples_count(distance: int, segments: list[list[int]]) -> int:
    sum = 0
    for s in segments:
        sum += max(0, get_possible_tuples_count_segment(distance, s))

    sum += get_neighbours_tuples_count(distance, segments)
    return sum

### Тестирование

In [17]:
def f(d, s):
    return get_possible_tuples_count_segment(d, s)

def F(d, ss):
    return get_possible_tuples_count(d, ss)

In [18]:
ss = [[1, 2, 3]]

assert(F(0, ss) == 2)
assert(F(1, ss) == 1)
assert(F(2, ss) == 0)

In [19]:
ss = [[1, 2, 3], [5, 6]]

assert(F(0, ss) == 3)
assert(F(1, ss) == 2)

In [20]:
ss = [[1, 2, 3], [5, 6, 7]]

assert(F(0, ss) == 4)
assert(F(1, ss) == 3)

In [21]:
ss = [[1], [5, 6, 7, 8], [10, 11, 12]]

assert (F(0, ss) == 5)
assert(F(1, ss) == 4)

In [22]:
ss = [[2], [5]]

assert (F(0, ss) == 0)
assert (F(1, ss) == 0)
assert (F(2, ss) == 1)
assert (F(3, ss) == 0)

### Индексация

In [23]:
d = {1: 3}
d

{1: 3}

In [24]:
l = [21, 43]
list2 = [1, 2, 3]

In [25]:
l + list2

[21, 43, 1, 2, 3]

In [2]:
from src.data.utils import get_possible_tuples

/home/nick/reidentification/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ss = [[1, 2, 3, 4, 5], [9, 10, 11, 12, 13], [23, 24, 25]]

In [4]:
get_possible_tuples(0, ss)

[(1, 2),
 (2, 3),
 (3, 4),
 (4, 5),
 (9, 10),
 (10, 11),
 (11, 12),
 (12, 13),
 (23, 24),
 (24, 25)]

In [5]:
get_possible_tuples(1, ss)

[(1, 3), (2, 4), (3, 5), (9, 11), (10, 12), (11, 13), (23, 25)]

In [6]:
get_possible_tuples(2, ss)

[(1, 4), (2, 5), (9, 12), (10, 13)]

In [8]:
get_possible_tuples(3, ss)

[(1, 5), (5, 9), (9, 13)]

In [9]:
get_possible_tuples(9, ss)

[(13, 23)]

In [22]:
for k, v in {1: 32, 3: 3}.items():
    print(v)

32
3
